# Clustering Consulting Project 

A large technology firm needs your help, they've been hacked! Luckily their forensic engineers have grabbed valuable data about the hacks, including information like session time,locations, wpm typing speed, etc. The forensic engineer relates to you what she has been able to figure out so far, she has been able to grab meta data of each session that the hackers used to connect to their servers. These are the features of the data:

* 'Session_Connection_Time': How long the session lasted in minutes
* 'Bytes Transferred': Number of MB transferred during session
* 'Kali_Trace_Used': Indicates if the hacker was using Kali Linux
* 'Servers_Corrupted': Number of server corrupted during the attack
* 'Pages_Corrupted': Number of pages illegally accessed
* 'Location': Location attack came from (Probably useless because the hackers used VPNs)
* 'WPM_Typing_Speed': Their estimated typing speed based on session logs.


The technology firm has 3 potential hackers that perpetrated the attack. Their certain of the first two hackers but they aren't very sure if the third hacker was involved or not. They have requested your help! Can you help figure out whether or not the third suspect had anything to do with the attacks, or was it just two hackers? It's probably not possible to know for sure, but maybe what you've just learned about Clustering can help!

**One last key fact, the forensic engineer knows that the hackers trade off attacks. Meaning they should each have roughly the same amount of attacks. For example if there were 100 total attacks, then in a 2 hacker situation each should have about 50 hacks, in a three hacker situation each would have about 33 hacks. The engineer believes this is the key element to solving this, but doesn't know how to distinguish this unlabeled data into groups of hackers.**

# EDA

In [1]:
import pyspark
spark = pyspark.sql.SparkSession.builder.appName("app").getOrCreate()

In [2]:
data = spark.read.csv(
    "/home/jovyan/udemy_pyspark/resources/Spark_for_Machine_Learning/Clustering/hack_data.csv",
    inferSchema=True,
    header=True,
)
data.printSchema()
data.describe().show()
data.head()

root
 |-- Session_Connection_Time: double (nullable = true)
 |-- Bytes Transferred: double (nullable = true)
 |-- Kali_Trace_Used: integer (nullable = true)
 |-- Servers_Corrupted: double (nullable = true)
 |-- Pages_Corrupted: double (nullable = true)
 |-- Location: string (nullable = true)
 |-- WPM_Typing_Speed: double (nullable = true)

+-------+-----------------------+------------------+------------------+-----------------+------------------+-----------+------------------+
|summary|Session_Connection_Time| Bytes Transferred|   Kali_Trace_Used|Servers_Corrupted|   Pages_Corrupted|   Location|  WPM_Typing_Speed|
+-------+-----------------------+------------------+------------------+-----------------+------------------+-----------+------------------+
|  count|                    334|               334|               334|              334|               334|        334|               334|
|   mean|     30.008982035928145| 607.2452694610777|0.5119760479041916|5.258502994011977|10.838323

Row(Session_Connection_Time=8.0, Bytes Transferred=391.09, Kali_Trace_Used=1, Servers_Corrupted=2.96, Pages_Corrupted=7.0, Location='Slovenia', WPM_Typing_Speed=72.37)

# Feature extraction

- drop location
- onehot kali ? 
- assemble
- scale afterwards

In [3]:
from pyspark.ml import Pipeline, Transformer
from pyspark.ml.feature import VectorAssembler, OneHotEncoder, StandardScaler

In [4]:
data.columns

['Session_Connection_Time',
 'Bytes Transferred',
 'Kali_Trace_Used',
 'Servers_Corrupted',
 'Pages_Corrupted',
 'Location',
 'WPM_Typing_Speed']

In [5]:
used_columns=[
    'Session_Connection_Time',
 'Bytes Transferred',
 # 'Kali_Trace_Used',
    'Kali_Cat',
 'Servers_Corrupted',
 'Pages_Corrupted',
 # 'Location',
 'WPM_Typing_Speed'
             ]

In [6]:
data = OneHotEncoder(inputCol="Kali_Trace_Used", outputCol="Kali_Cat"
                    ).fit(data).transform(data)
data.head()

Row(Session_Connection_Time=8.0, Bytes Transferred=391.09, Kali_Trace_Used=1, Servers_Corrupted=2.96, Pages_Corrupted=7.0, Location='Slovenia', WPM_Typing_Speed=72.37, Kali_Cat=SparseVector(1, {}))

In [7]:
data = VectorAssembler(inputCols=used_columns, outputCol="features"
                      ).transform(data)
data.head()

Row(Session_Connection_Time=8.0, Bytes Transferred=391.09, Kali_Trace_Used=1, Servers_Corrupted=2.96, Pages_Corrupted=7.0, Location='Slovenia', WPM_Typing_Speed=72.37, Kali_Cat=SparseVector(1, {}), features=DenseVector([8.0, 391.09, 0.0, 2.96, 7.0, 72.37]))

In [8]:
data = StandardScaler(inputCol="features", outputCol="scaled_features"
                     ).fit(data).transform(data)
data.head()

Row(Session_Connection_Time=8.0, Bytes Transferred=391.09, Kali_Trace_Used=1, Servers_Corrupted=2.96, Pages_Corrupted=7.0, Location='Slovenia', WPM_Typing_Speed=72.37, Kali_Cat=SparseVector(1, {}), features=DenseVector([8.0, 391.09, 0.0, 2.96, 7.0, 72.37]), scaled_features=DenseVector([0.5679, 1.3658, 0.0, 1.2859, 2.2849, 5.3963]))

# Train

In [9]:
from pyspark.ml.clustering import KMeans

In [10]:
kmeans = KMeans(featuresCol='scaled_features',k=2)
model = kmeans.fit(data)

In [11]:
results = model.transform(data)
results.head()

Row(Session_Connection_Time=8.0, Bytes Transferred=391.09, Kali_Trace_Used=1, Servers_Corrupted=2.96, Pages_Corrupted=7.0, Location='Slovenia', WPM_Typing_Speed=72.37, Kali_Cat=SparseVector(1, {}), features=DenseVector([8.0, 391.09, 0.0, 2.96, 7.0, 72.37]), scaled_features=DenseVector([0.5679, 1.3658, 0.0, 1.2859, 2.2849, 5.3963]), prediction=0)

In [12]:
results.select("prediction").groupby("prediction").count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|         1|  167|
|         0|  167|
+----------+-----+



Using k=3 we get

```
+----------+-----+
|prediction|count|
+----------+-----+
|         1|   79|
|         2|   88|
|         0|  167|
+----------+-----+
```

Using k=2 we get

```
+----------+-----+
|prediction|count|
+----------+-----+
|         1|  167|
|         0|  167|
+----------+-----+
```

Based on the hypothesis that each hacker has to have the same number of attacks, it's more likely that there were two hackers instead of three.

# Evaluation

In [13]:
# this worked in the code along example, but not now.
# Probably need an evaluator again, see 
model.computeCost

AttributeError: 'KMeansModel' object has no attribute 'computeCost'

In [ ]:
from pyspark.ml.evaluation import ClusteringEvaluator
ClusteringEvaluator(
    distanceMeasure="squaredEuclidean",
).evaluate(results)